# 2024 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [1]:
## import libraries

!pip3 install torchtext==0.4.0

# Standard libraries
import math
import string
import pandas as pd
import numpy as np
import json

# Machine learning and deep learning libraries
import tensorflow
import keras
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.data import Field, LabelField, Example, Dataset
from torchtext.data.utils import get_tokenizer

# Natural Language Processing (NLP) libraries
import nltk
import spacy
from nltk.corpus import stopwords

# Visualization
import matplotlib.pyplot as plt

In [2]:
## reading in json files

"""
Description of json files
* [train-claims,dev-claims].json: JSON files for the labelled training and development set; 
* evidence.json: JSON file containing a large number of evidence passages (i.e. the “knowledge source”); 
* dev-claims-baseline.json: JSON file containing predictions of a baseline system on the development set;
"""

## relative file paths

## baseline system - will not be used for any training/evaluation
devClaimsBaselineFile='./data/dev-claims-baseline.json'
## use this for model training
trainClaimsFile='./data/train-claims.json'
## use this set for hyperparameter tuning and evaluation metric 
devClaimsFile='./data/dev-claims.json'
## evidence files need to be downloaded through https://drive.google.com/file/d/1JlUzRufknsHzKzvrEjgw8D3n_IRpjzo6/view?usp=sharing as it is to big to be uploaded to github
evidenceFile='./data/evidence.json'

testFile='./data/test-claims-unlabelled.json'

tokenizer=get_tokenizer("basic_english")


In [3]:

# Define tokenizer
tokenizer=get_tokenizer("basic_english")
stopWords=set(stopwords.words('english'))
punctuation=string.punctuation

def tokenizeAndStopwordRemoval(text):
    tokens=tokenizer(text)
    filteredTokens=[word for word in tokens if (word not in stopWords) and (word not in punctuation)]
    return filteredTokens


TEXT=torchtext.data.Field(tokenize=tokenizeAndStopwordRemoval,
                          init_token='<sos>',
                          eos_token='<sos>',
                          lower=True)
LABEL = torchtext.data.LabelField(dtype=torch.float)

# Load datasets
def load_json_data(json_file, text_field, label_field):
    with open(json_file, 'r') as file:
        data=json.load(file)
    examples=[]
    for i,claimText in data.items():
        text=claimText['claim_text']
        label=claimText['evidences']
        examples.append(Example.fromlist([text, label], fields=[('text', text_field), ('label', label_field)]))
    return examples

def load_json_data_test(json_file, text_field):
    with open(json_file, 'r') as file:
        data=json.load(file)
    examples=[]
    for i,claimText in data.items():
        text=claimText['claim_text']
        examples.append(Example.fromlist([text], fields=[('text', text_field)]))
    return examples

train_examples=load_json_data(trainClaimsFile, TEXT, LABEL)
dev_examples=load_json_data(devClaimsFile, TEXT, LABEL)
test_examples=load_json_data_test(testFile,TEXT)

train_dataset = Dataset(train_examples, fields=[('text', TEXT), ('label', LABEL)])
dev_dataset = Dataset(dev_examples, fields=[('text', TEXT), ('label', LABEL)])
test_dataset=Dataset(dev_examples, fields=[('text', TEXT)])

for i, example in enumerate(dev_dataset.examples):
    if i < 5:
        print(vars(example))

# Build vocabulary
TEXT.build_vocab(train_dataset)
LABEL.build_vocab(train_dataset)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Batchify function
def batchify(dataset, bsz):
    data = TEXT.process([getattr(x, 'text') for x in dataset.examples])
    nbatch = data.size(0) // bsz
    data = data.narrow(0, 0, nbatch * bsz)
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

TEST = Field(tokenize=tokenizeAndStopwordRemoval, lower=True, init_token='<sos>', eos_token='<eos>', batch_first=True)
TEST.build_vocab(train_dataset)

def processTextOnly(dataset):
    data = TEST.process([getattr(x, 'text') for x in dataset.examples])
    return data.to(device)

# Batch sizes
batch_size = 20
eval_batch_size = 10

# Process the datasets

train_data = batchify(train_dataset, batch_size)
dev_data = batchify(dev_dataset, eval_batch_size)
test_data=processTextOnly(test_dataset)
dev_accuracy_data=processTextOnly(dev_dataset)

bptt = 35
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

{'text': ['[south', 'australia]', 'expensive', 'electricity', 'world'], 'label': ['evidence-67732', 'evidence-572512']}
{'text': ['3', 'per', 'cent', 'total', 'annual', 'global', 'emissions', 'carbon', 'dioxide', 'humans', 'australia', 'prod\xaduces', '1', '3', 'per', 'cent', '3', 'per', 'cent', 'amount', 'emissions', 'reductio\xadn', 'effect', 'global', 'climate'], 'label': ['evidence-996421', 'evidence-1080858', 'evidence-208053', 'evidence-699212', 'evidence-832334']}
{'text': ['means', 'world', '1c', 'warmer', 'pre-industrial', 'times'], 'label': ['evidence-889933', 'evidence-694262']}
{'text': ['“as', 'happens', 'zika', 'may', 'also', 'good', 'model', 'second', 'worrying', 'effect', '—', 'disease', 'mutation'], 'label': ['evidence-422399', 'evidence-702226', 'evidence-286834', 'evidence-472751', 'evidence-641043']}
{'text': ['greenland', 'lost', 'tiny', 'fraction', 'ice', 'mass'], 'label': ['evidence-52981', 'evidence-264761', 'evidence-947243', 'evidence-424102']}


# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [4]:
class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        #triu returns the upper triangular part of a matrix (2-D tensor) or batch of matrices (see section below)
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return output

In [5]:
#triu returns the upper triangular part of a matrix (2-D tensor) or batch of matrices (see section below)
torch.triu(torch.ones(3, 3))

tensor([[1., 1., 1.],
        [0., 1., 1.],
        [0., 0., 1.]])

In [6]:
# Masking
def masking():
  sz = 4
  mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
  mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))

  return mask

masking()

tensor([[0., -inf, -inf, -inf],
        [0., 0., -inf, -inf],
        [0., 0., 0., -inf],
        [0., 0., 0., 0.]])

In [7]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term) #0::2 means starting with index 0, step = 2
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)
    
ntokens = len(TEXT.vocab.stoi) # the size of vocabulary
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)


c:\Users\julev\anaconda3\envs\Python-3-8\lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [70]:
criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    ntokens = len(TEXT.vocab.stoi)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()


def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    total_correct = 0
    total_samples = 0
    ntokens = len(TEXT.vocab.stoi)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            output = eval_model(data)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
            
            # Calculate accuracy
            _, predicted = torch.max(output_flat, 1)
            total_correct += (predicted == targets).sum().item()
            total_samples += targets.size(0)

    loss = total_loss / (len(data_source) - 1)
    accuracy = total_correct / total_samples
    return loss, accuracy

best_val_loss = float("inf")
epochs = 100 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss,accuracy = evaluate(model, dev_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('ACCURACY',accuracy)
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()


-----------------------------------------------------------------------------------------
| end of epoch   1 | time:  1.97s | valid loss  3.80 | valid ppl    44.59
ACCURACY 0.6164227642276423
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   2 | time:  2.15s | valid loss  4.01 | valid ppl    55.18
ACCURACY 0.6138211382113821
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   3 | time:  2.04s | valid loss  4.61 | valid ppl   100.14
ACCURACY 0.5489430894308943
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   4 | time:  2.10s | valid loss  4.55 | valid ppl

In [66]:
test_loss = evaluate(best_model, dev_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

model.eval()
output=model(dev_data)
probabilities=torch.nn.functional.softmax(output,dim=-1)
_,predictedIndex=torch.max(probabilities,dim=-1)
# for i in predictedIndex:
#     print(predictedIndex[i])

# print(LABEL.vocab.stoi)
from sklearn.metrics import accuracy_score

# vocab = LABEL.vocab.stoi 
# reverseVocab = {index: word for word, index in vocab.items()} 
# def tensor_to_text(tensor, index_to_word):
#     words = [index_to_word[index.item()] for index in tensor if index in index_to_word]
#     return ' '.join(words)

# # print(reverseVocab)

# for i in predictedIndex:
#     #print(i)
#     words = list(set([reverseVocab[index.item()] for index in i]))
#     print(words)


#accuracyScore=accuracy_score()

| End of training | test loss  3.88 | test ppl    48.62


In [67]:
## import as pandas dataframe
devClaimsBaseline=pd.read_json(devClaimsBaselineFile)
trainClaims=pd.read_json(trainClaimsFile)
devClaims=pd.read_json(devClaimsFile)
evidence=pd.read_json(evidenceFile,orient='index')
evidences=pd.DataFrame(list(evidence.items()), columns=['evidence_id', 'evidence_text'])

## Separate claim_text,claim_label, and evidences from training and development sets, saved as pd dataframes
claimTextTrain=trainClaims.loc['claim_text'].to_frame()
claimLabelTrain=trainClaims.loc['claim_label'].to_frame()
evidenceTrain=trainClaims.loc['evidences'].to_frame()

claimTextDev=devClaims.loc['claim_text'].to_frame()
claimLabelDev=devClaims.loc['claim_label'].to_frame()
evidenceDev=devClaims.loc['evidences'].to_frame()


In [68]:

pred=model(test_data)
print(pred)
import torch

# use softmax
probabilities = torch.softmax(pred, dim=-1)
_, predicted_indices = torch.max(probabilities, dim=-1)
predicted_words = [[LABEL.vocab.itos[index] for index in example] for example in predicted_indices]
predicted_text={}
# Print the words
for i, sentence in enumerate(predicted_words):
    print(f"Sentence {i+1}: {' '.join(sentence)}")
    predicted_text[f'Sentence{i}']={', '.join(sentence)}


remove_tokens = ['<pad>', '<unks>', '<sos>', '<eos>']
def clean_sentence(sentence, tokens_to_remove):
    for token in tokens_to_remove:
        sentence = sentence.replace(token, '')
    return sentence.strip()
cleaned_data = {key: [clean_sentence(sentence, remove_tokens) for sentence in value] for key, value in predicted_text.items()}
print(cleaned_data)

# vectorizer = TfidfVectorizer()
# tfidf_matrix = vectorizer.fit_transform(evidenceDataframe['combined_evidence'])
# print(tfidf_matrix)

unique_words_per_sentence = {}
for key, value in cleaned_data.items():
    # Split by spaces and filter out numeric values and punctuation
    words = set(word.strip('–,.') for word in value[0].split())
    unique_words_per_sentence[key] = words

print(unique_words_per_sentence)

tensor([[[-2.2244e+00,  1.4384e+01,  2.0575e+01,  ...,  3.4384e-01,
           1.7164e-01,  7.2193e-01],
         [-1.3240e+00,  8.5455e+00,  7.9217e+00,  ..., -1.6499e+00,
           5.7886e-01, -1.9355e+00],
         [-1.3240e+00,  8.5455e+00,  7.9217e+00,  ..., -1.6499e+00,
           5.7886e-01, -1.9355e+00],
         ...,
         [-8.0851e-01,  1.1719e+01,  8.4655e+00,  ..., -3.7274e-03,
          -5.3798e-01,  2.0174e-01],
         [-8.0851e-01,  1.1719e+01,  8.4655e+00,  ..., -3.7274e-03,
          -5.3798e-01,  2.0174e-01],
         [-8.0851e-01,  1.1719e+01,  8.4655e+00,  ..., -3.7274e-03,
          -5.3798e-01,  2.0174e-01]],

        [[-2.2145e+00,  1.4368e+01,  2.0605e+01,  ...,  3.3397e-01,
           1.8118e-01,  6.9826e-01],
         [-1.4297e+00,  8.5009e+00,  8.6010e+00,  ..., -1.9297e+00,
           7.0071e-01, -2.4352e+00],
         [-1.4297e+00,  8.5009e+00,  8.6010e+00,  ..., -1.9297e+00,
           7.0071e-01, -2.4352e+00],
         ...,
         [-8.0861e-01,  1

In [69]:
predictions_df = pd.DataFrame(list(cleaned_data.items()), columns=['Sentence', 'PredictedEvidences'])

# Resetting index on evidenceDev without keeping the old index
evidenceDev_reset = evidenceDev.reset_index(drop=True)

# Resetting index on predictions_df without keeping the old index
predictions_df_reset = predictions_df.reset_index(drop=True)

# Merging the DataFrames on their new integer indices
combined_df = pd.merge(evidenceDev_reset, predictions_df_reset['PredictedEvidences'], left_index=True, right_index=True, how='outer')

print(combined_df)
def calculate_metrics(df):
    true_positive = 0
    false_positive = 0
    false_negative = 0
    exact_match = 0
    for _, row in df.iterrows():
        true_set = set(row['evidences'])
        pred_set = set(row['PredictedEvidences'])
        if true_set == pred_set:
            exact_match += 1
        tp = len(true_set & pred_set)
        fp = len(pred_set - true_set)
        fn = len(true_set - pred_set)
        
        true_positive += tp
        false_positive += fp
        false_negative += fn
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = exact_match / len(df)
    return {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1_score
    }

# Calculate and print metrics
metrics = calculate_metrics(combined_df)
print(metrics)

                                             evidences  \
0                    [evidence-67732, evidence-572512]   
1    [evidence-996421, evidence-1080858, evidence-2...   
2                   [evidence-889933, evidence-694262]   
3    [evidence-422399, evidence-702226, evidence-28...   
4    [evidence-52981, evidence-264761, evidence-947...   
..                                                 ...   
149  [evidence-409365, evidence-127519, evidence-85...   
150  [evidence-368192, evidence-261690, evidence-20...   
151  [evidence-1124018, evidence-995813, evidence-1...   
152                                  [evidence-660755]   
153                [evidence-242575, evidence-1175280]   

                                    PredictedEvidences  
0    [evidence-1120350, evidence-618094, evidence-6...  
1    [evidence-1120350, evidence-618094, evidence-6...  
2    [evidence-1120350, evidence-1127398, evidence-...  
3    [evidence-1120350, evidence-498301, evidence-4...  
4    [evidence-112

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*